In [1]:
!pip install dspy-ai
!pip install langchain_community
!pip install clarifai
!pip install langchain

In [2]:
import dspy
import numpy as np
from dspy.retrieve.clarifai_rm import ClarifaiRM 
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import Clarifai as clarifaivectorstore
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
llama3 = dspy.OllamaLocal(model='llama3', timeout_s = 10000)
dspy.settings.configure(lm=llama3)

In [4]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
embedding_model = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-large-en",
                               model_kwargs={'device': 'cpu'},
                               encode_kwargs={'normalize_embeddings': False})

C:\Users\VISHAL MAURYA\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
# !pip install fastembed
# from langchain_community.embeddings import FastEmbedEmbeddings
# embedding_model = FastEmbedEmbeddings(model_name = "BAAI/bge-large-en-v1.5")

In [6]:
def embed_text(text):
    # embedding = FastEmbedEmbeddings.embed_query(embedding_model,text = text)
    embedding = HuggingFaceBgeEmbeddings.embed_query(embedding_model,text = text)
    return embedding

# PDF

In [ ]:
PDF_PATH = "Documents - Renamed/US Lighting Installation.pdf"
docs = PyPDFLoader(PDF_PATH).load()
text = '\n'.join([doc.page_content for doc in docs])

In [ ]:
# Direct call 
s = time.time()
print(llama3(prompt)[0])
print(s-time.time())

In [35]:
# ChainOfThought
cot = dspy.ChainOfThought('question -> answer') #n = 5

s = time.time()
response = cot(question=prompt)#.completions.answer
print(time.time() - s)

817.4924392700195


In [37]:
response

Prediction(
    rationale='Based on the provided text, I will attempt to answer your question.\n\n**Answer:** The scope of this PDF appears to be a set of regulations and guidelines for the approval of vehicle lighting systems, specifically related to dipped-beam headlamps and their automatic switching conditions.',
    answer='Here is the generated field `answer`:\n\n**Answer:** The scope of this PDF appears to be a set of regulations and guidelines for the approval of vehicle lighting systems, specifically related to dipped-beam headlamps and their automatic switching conditions.'
)

# Question-Answer Dataset

In [7]:
dataset = [dspy.Example(question="Explain Contour Marking.", answer="Contour marking is a type of conspicuity marking that is intended to indicate the horizontal and vertical dimensions (length, width, and height) of a vehicle. It helps to make the vehicle more visible to other road users by highlighting its overall shape and size. There are two types of contour markings: full contour marking and partial contour marking. A full contour marking indicates the outline of the vehicle by a continuous line, while a partial contour marking indicates the horizontal dimension of the vehicle by a continuous line and the vertical dimension by marking the upper corners. In the case of a partial contour marking, each upper corner is described by two lines at 90Â° to each other, each at least 250 mm in length."),
           dspy.Example(question="What is the different regulations mentioned in standard?", answer="The standard mentions several regulations, including:1. Regulation No. 4 2. Regulation No. 98 (prior to Supplement 9) 3. Regulation No. 112 (prior to Supplement 8) 4. This Regulation (referring to the current regulation being discussed) 5. The 1958 Agreement Additionally, the standard also references various series of amendments to these regulations, including: 1. 03 series of amendments 2. 04 series of amendments 3. 05 series of amendments 4. 06 series of amendments"),
           dspy.Example(question="Can an AFS be used as a passing beam?", answer="An AFS (Adaptive Front Lighting System) can be used as a passing beam, but with certain conditions and limitations. According to section 6.22.7.2, the dipped-beam may remain switched on at the same time as the main-beams, and the control for changing over to the dipped-beam shall switch off all main-beam headlamps or de-activate all AFS lighting units for the main-beam simultaneously, implying that the AFS can provide passing-beam lighting. Furthermore, section 6.22.7.4 specifies that the passing beam function of the AFS is divided into different classes (C, V, E, and W) and modes, each with its own set of activation conditions and requirements. For instance, the class C mode of the passing beam shall be activated if no mode of another passing-beam class is activated, while the class V mode shall not operate unless certain conditions such as road type and vehicle speed are met."),           
           dspy.Example(question="Can an AFS used as a main beam?", answer="Yes, an AFS (Adaptive Front-lighting System) can be used as a main beam. According to section 6.22.7.1.1, the lighting units for the main-beam, including AFS, can be activated either simultaneously or in pairs, allowing for the use of AFS as a main beam. Additionally, section 6.22.7.2. specifies that when changing over to the dipped-beam, all main-beam headlamps or AFS lighting units for the main-beam must be switched off or deactivated simultaneously, further supporting the use of AFS as a main beam."),  
           dspy.Example(question="Can a red light be placed in front of the vehicle?", answer="According to section 5.10.1, a red light cannot be placed in front of a vehicle, with the exception of a red rearmost side-marker lamp. This is because there shall be no direct visibility of the apparent surface of a red lamp if viewed by an observer moving within Zone 1 as specified in Annex 4."),
           dspy.Example(question="Can a White light placed in the rear of the vehicle?", answer="A white light cannot be placed in the rear of the vehicle. According to section 5.10.2, there shall be no direct visibility of the apparent surface of a white lamp if viewed by an observer moving within Zone 2 in a transverse plane situated 25 m behind the vehicle, implying that white light should not be visible from the rear of the vehicle. Additionally, section 6.10.6 specifies that the orientation of the rear position lamp is 'Rearwards', suggesting that the lamp should emit a red light, not white. Furthermore, section 5 states that conspicuity marking should be red or yellow to the rear, and section 6.14 specifies that the rear retro-reflector shall conform to the requirements concerning Class IA or IB retro-reflectors in Regulation No. 3, which likely do not allow white light. Therefore, based on the provided context, a white light placed in the rear of the vehicle is not permitted."),
           dspy.Example(question="Which lamps need to indicate flashing warning light in Tell-tale?", answer="According to the regulations, the lamps that need to indicate a flashing warning light in the Tell-tale are the direction-indicator lamps and the hazard warning signal. The direction-indicator lamps require a flashing warning light in the Tell-tale to indicate malfunction or failure, as specified in Section 6.5.8. Similarly, the hazard warning signal also requires a flashing circuit-closed tell-tale, as mandated in Section 6.6.8."),
           dspy.Example(question="What is the condition for switching on the Reversing lamp?", answer="The condition for switching on the Reversing lamp is that the lamp can light up only if the reverse gear is engaged and if the device which controls the starting and stopping of the engine is in such a position that operation of the engine is possible."),
           dspy.Example(question="What is the minimum distance of side direction indicator from the rear part 38 of the vehicle?", answer="Unfortunately, there is no specific information provided about the minimum distance of the side direction indicator from the rear part of the vehicle. The provided sections (6.10.4.1 and 6.5.4.3) discuss the distance between the inner edges of the apparent surfaces and the forward boundary of the vehicle's overall length, respectively, but do not mention the minimum distance from the rear part of the vehicle. Therefore, it is not possible to provide a specific minimum distance in this case."),
           dspy.Example(question="Can a rear fog lamp be switched ON by switching ON any other lamps?", answer="A rear fog lamp cannot be switched on simply by switching on any other lamp, except for the main beams, dipped-beams, or front fog-lamps. According to paragraph 6.11.7.1, the rear fog lamp(s) cannot be switched on unless one of these specific lamps is already lit. This means that switching on other lamps, such as parking lamps or side marker lamps, will not activate the rear fog lamp. However, there are additional exceptions mentioned in paragraphs 6.11.7.3 and 6.11.7.5, which may allow the rear fog lamp to be switched on under certain conditions.")
          ]
dataset = [x.with_inputs('question') for x in dataset]
trainset = [dataset[i] for i in range(10) if i % 2 == 0]
devset = [dataset[i] for i in range(10) if i % 2 != 0]

In [7]:
class GenerateAnswer(dspy.Signature):
    """Think and Answer questions based on the context provided."""
    question = dspy.InputField(desc = "question to be answer")
    answer = dspy.OutputField(desc="generate detailed answer")

In [8]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        return self.generate_answer(question=question)

In [9]:
cot = CoT()

In [17]:
# query = "what is scope of the pdf?"
query = "What are the lights and lamps used in vehicle?"
query = trainset[0].question
prompt = f"""Answer the question only from the given context. Answer should be accurate and in detail.
CONTEXT : {text}
QUESTION : {query}"""
cot.forward(question = prompt)

Prediction(
    rationale='Based on the provided context, I will answer your question.\n\nQuestion: Explain Contour Marking.\n\nReasoning: Since there is no specific mention of "Contour Marking" in the given text, it seems that this term might not be directly related to the regulations or standards mentioned. However, considering the context and the format of the provided information (Annexes 13-15), I would assume that Contour Marking refers to a visual marking or indication on the vehicle\'s exterior, possibly used for safety purposes.\n\nIn this case, it could be related to the zones of observation mentioned in Annex 14, which defines the observing areas towards the apparent surface of maneuvering and courtesy lamps. The contour marking might serve as a visual aid',
    answer='Based on the provided context, I would answer:\n\nContour Marking refers to a visual marking or indication on the vehicle\'s exterior, possibly used for safety purposes. It could be related to the zones of ob

# Question-Context-Answer Dataset

In [16]:
dataset2 = [
           dspy.Example(question = "Explain Contour Marking.", context = text, answer = "Contour marking is a type of conspicuity marking that is intended to indicate the horizontal and vertical dimensions (length, width, and height) of a vehicle. It helps to make the vehicle more visible to other road users by highlighting its overall shape and size. There are two types of contour markings: full contour marking and partial contour marking. A full contour marking indicates the outline of the vehicle by a continuous line, while a partial contour marking indicates the horizontal dimension of the vehicle by a continuous line and the vertical dimension by marking the upper corners. In the case of a partial contour marking, each upper corner is described by two lines at 90Â° to each other, each at least 250 mm in length."),
           dspy.Example(question = "Can a red light be placed in front of the vehicle?", context = text, answer = "According to section 5.10.1, a red light cannot be placed in front of a vehicle, with the exception of a red rearmost side-marker lamp. This is because there shall be no direct visibility of the apparent surface of a red lamp if viewed by an observer moving within Zone 1 as specified in Annex 4."),
           dspy.Example(question = "What does headlamp levelling mean?", context = text, answer = "Headlamp levelling refers to the adjustment of the vertical inclination of the headlamp beam to ensure that it remains within the prescribed limits under various static loading conditions, providing optimal illumination of the road ahead while minimizing glare to oncoming traffic. This is achieved through a levelling device that automatically adjusts the inclination of the headlamp beam to compensate for changes in the vehicle's load, ensuring that the beam remains within the allowed range and preventing undue dazzle or discomfort to oncoming drivers and other road-users. The purpose of headlamp levelling is to maintain a safe and comfortable lighting pattern on the road, taking into account the vehicle's load and other factors."),
           dspy.Example(question = "What are the requirements for O category of vehicle?", context = text, answer = """The requirements for O category of vehicles are as follows:

* Two rear direction-indicator lamps (Categories 2a or 2b) are optional (Paragraph 6.22.5.).
* A maximum of three optional category 5 or one optional category 6 device per side on vehicles of type O, O2, and O3 exceeding 9 m in length (Paragraph 6.22.5.).
* Two optional category S1 or S2 devices may be installed on vehicles in categories O, O2, and O3 (Paragraph 6.7.2.1.).

These requirements are specific to the lighting and light-signalling devices of O category vehicles."""),
           dspy.Example(question = "What are the requirements of the Off-road vehicle?", context = text, answer = """The requirements of an Off-road vehicle, specifically category N G, include:

* The maximum height of the light may be increased to 1,500 mm (section 6.2.4.2).
* For headlamps exceeding a height of 1,200 mm, the limits for the vertical inclination of the cut-off shall be between -1.5 per cent and -3.5 per cent, and the initial aim shall be set between -2 per cent and -2.5 per cent (section 6.2.6.1.2)."""),
           dspy.Example(question = "What are the different light sources and explain them?", context = text, answer = """There are several types of light sources, each with its own unique characteristics and applications. These include:

1. **Filament light source**: A light source where the element for visible radiation is one or more heated filaments producing thermal radiation.

2. **Gas-discharge light source**: A light source where the element for visible radiation is a discharge arc producing electro-luminescence/fluorescence.

3. **Light-emitting diode (LED) light source**: A light source where the element for visible radiation is one or more solid state junctions producing injection-luminescence/fluorescence.

4. **LED module**: A light source module containing as light sources only LEDs, which may optionally contain one or more holders for approved replaceable light sources.

In addition to these, there are also light sources with different optical configurations, including:

5. **Light source with a reflector optic behind an outer lens**: This type of light source has a reflector optic behind an outer lens, which can be either included or excluded from the apparent surface.

6. **Light source with a reflector optic with an inner lens behind an outer lens**: This type of light source has a reflector optic with an inner lens behind an outer lens, which can also be either included or excluded from the apparent surface.

7. **Light source with a reflector optic with a partial inner lens behind an outer lens**: This type of light source has a reflector optic with a partial inner lens behind an outer lens, which can also be either included or excluded from the apparent surface.

8. **Light guide optic behind an outer lens**: This type of light source has a light guide optic behind an outer lens, which can be either included or excluded from the apparent surface.

9. **Light guide optic or reflector optic behind an outer lens**: This type of light source has either a light guide optic or a reflector optic behind an outer lens, which can be either included or excluded from the apparent surface.

10. **Light source with a reflector optic in combination with an area which is not part of this function, behind an outer lens**: This type of light source has a reflector optic in combination with an area that is not part of the light source function, behind an outer lens, which can be either included or excluded from the apparent surface.

These different light sources are used in various applications, including lamps and light-signalling devices, and are designed to provide specific types of illumination or light signals to other road users."""),
           dspy.Example(question = "Is Filament light source and Gas-Discharge light source are different?", context = text, answer = "Yes, Filament light source and Gas-Discharge light source are different. A Filament light source is defined as a light source where the element for visible radiation is one or more heated filaments producing thermal radiation. On the other hand, a Gas-Discharge light source is defined as a light source where the element for visible radiation is a discharge arc producing electro-luminescence/fluorescence. These definitions indicate that they are two distinct types of light sources with different operating principles and characteristics."),
           dspy.Example(question = "Can a red light be placed in front of the vehicle?", context = text, answer = "According to section 5.10.1, a red light cannot be placed in front of a vehicle, with the exception of a red rearmost side-marker lamp. This is because there shall be no direct visibility of the apparent surface of a red lamp if viewed by an observer moving within Zone 1 as specified in Annex 4."),
           dspy.Example(question = "What are the different light colors that can be used in vehicles?", context = text, answer = """The different light colors that can be used in vehicles are:

1. White: Used for main-beam headlamps, dipped-beam headlamps, front fog lamps, reversing lamps, front position lamps, rear registration plate lamps, exterior courtesy lamps, manoeuvring lamps, adaptive front-lighting systems (AFS), cornering lamps, and retro-reflectors (if colourless).

2. Selective Yellow: Used for front fog lamps.

3. Amber: Used for direction-indicator lamps, hazard warning signals, parking lamps (if reciprocally incorporated in the side direction-indicator lamps or in the side-marker lamps), and side-marker lamps.

4. Red: Used for stop lamps, rear position lamps, rear fog lamps, rear-end collision alert signals, and rear rearmost side-marker lamps.

5. Yellow: Used for rear fog lamps, rear rearmost side-marker lamps, and conspicuity markings to the side (if not white).

Note that some colors, like blue, are mentioned in the context as boundaries for white or selective yellow, but are not explicitly mentioned as colors used for specific lamps. Additionally, yellowish green, yellowish white, reddish purple, and purple are defined as chromaticity areas but are not explicitly mentioned as colors used for specific lamps."""),
           dspy.Example(question = "Explain different types of Lighting mode?", context = text, answer = """There are various types of lighting modes, which can be categorized into different groups based on their functions and purposes. Here are some of the main types of lighting modes:

**Headlamp Modes**:

* Main-beam headlamps: high-intensity lamps used for driving at night or in low-light conditions.
* Dipped-beam headlamps: low-intensity lamps used for driving at night or in low-light conditions, designed to illuminate the road ahead without dazzling oncoming traffic.
* Adaptive front lighting system (AFS): a system that adjusts the beam pattern of the headlamps to suit different driving conditions.

**Fog and Special Conditions Modes**:

* Front-fog lamps: lamps used to improve visibility in foggy or misty conditions.
* Rear fog-lamps: lamps used to improve visibility in foggy or misty conditions from the rear of the vehicle.
* Cornering lamps: lamps used to improve visibility when cornering or turning.
* Manoeuvring lamps: lamps used to improve visibility when manoeuvring the vehicle.

**Indicator and Warning Modes**:

* Direction-indicators: lamps used to indicate the direction of travel or intended direction of travel.
* Hazard warning signal: a lamp or lamps that flash to indicate a hazard or warning to other road users.
* Stop-lamps: lamps used to indicate when the vehicle is braking or stopping.

**Position and Presence Modes**:

* Front position lamps: lamps used to indicate the presence and position of the vehicle from the front.
* Rear position lamps: lamps used to indicate the presence and position of the vehicle from the rear.
* Rear registration plate illuminating device: a lamp used to illuminate the rear registration plate.
* End-outline marker lamps: lamps used to indicate the outline of the vehicle.

**Convenience and Safety Modes**:

* Parking lamps: lamps used to indicate the presence and position of the vehicle when parked.
* Daytime running lamps: lamps used to increase the visibility of the vehicle during daylight hours.
* Exterior courtesy lamps: lamps used to provide additional lighting when entering or exiting the vehicle.
* Retro-reflectors: devices that reflect light back to its source, used to increase the visibility of the vehicle.

**Variable Luminous Intensity Control**:

* Certain lamps, such as rear direction-indicator lamps, rear position lamps, stop lamps, and rear fog lamps, are allowed to have variable luminous intensity control, which can respond to external influences such as ambient lighting, fog, snowfall, rain, spray, dust clouds, or contamination of the light-emitting surface.

These lighting modes are designed to provide improved safety, visibility, and convenience for drivers and other road users, and are subject to specific regulations and requirements governing their use."""),
           dspy.Example(question = "Difference between passing beam and dipped beam?", context = text, answer = """The difference between passing beam and dipped beam lies in their functionality, operation, and purpose. The passing beam, also referred to as Adaptive Front-lighting System (AFS), is an advanced system that automatically adjusts the beam pattern to provide better illumination of the road ahead, taking into account various factors such as speed, road conditions, and ambient light. It can switch between different modes depending on the driving conditions and may include features like bend lighting, where the beam is adjusted to follow the direction of the road.

On the other hand, the dipped beam is a lower beam of light used for normal driving conditions, providing a shorter and wider beam pattern compared to the main beam. It is typically used for urban driving, where the beam is directed downwards to reduce glare for oncoming traffic. The dipped beam may be adjusted automatically or manually, depending on the vehicle's system.

In essence, the main beam (driving beam) is used for long-distance illumination, while the dipped beam (passing beam) is used for shorter-distance illumination without causing glare to oncoming traffic. The passing beam is a more advanced, adaptive system that adjusts to suit various driving conditions, whereas the dipped beam is a more traditional, lower beam of light used for everyday driving."""),
           dspy.Example(question = "For which light Visual alert signal is required?", context = text, answer = "A visual alert signal is required for the hazard warning signal, the rear-end collision alert signal, and the direction-indicator lamps of categories 1, 1a, 1b, 2a, and 2b."),
           dspy.Example(question = "What are the lights to be placed in Rear of the vehicle?", context = text, answer = """The lights to be placed at the rear of the vehicle are:

1. Rear position lamp (Section 6.10)
2. Rear direction-indicator (Section 6.10)
3. Rear retro-reflector, triangular (Section 5.18)
4. Rear retro-reflector, non-triangular (Sections 5.18 and 6.15)
5. Rear fog lamp (Sections 6.10.6 and 6.11)
6. Stop-lamp (Sections 6.10.6 and 6.11.9)
7. Red rearmost side-marker lamp (Section 6.10.6)
8. Rear end-outline marker lamp (Section 6.13)

Note that each of these lights has specific requirements or restrictions, such as orientation, number, arrangement, and position, which can be found in the corresponding sections of the context.""")
          ]
dataset2 = [x.with_inputs('question', 'context') for x in dataset2]
# trainset2 = [dataset2[i] for i in range(13) if i % 2 == 0]
# devset2 = [dataset2[i] for i in range(13) if i % 2 != 0]

In [12]:
class GenerateAnswer2(dspy.Signature):
    """Think and Answer questions based on the context provided."""
    context = dspy.InputField(desc="contains relevant context")
    question = dspy.InputField(desc = "given question to be answer")
    answer = dspy.OutputField(desc="generate detailed answer")

In [13]:
class CoT2(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer2 = dspy.ChainOfThought(GenerateAnswer2)
    
    def forward(self, question, context):
        context = context
        return self.generate_answer2(context=context,question=question)

In [14]:
def metric(example, pred, trace=None):
    a = np.array(embed_text(example.answer)).reshape(1, -1)
    b = np.array(embed_text(pred.answer)).reshape(1, -1)
    return cosine_similarity(a,b)

In [15]:
from dspy.teleprompt import BootstrapFewShot

optimizer = BootstrapFewShot(metric = metric, max_bootstrapped_demos=13, max_labeled_demos=13)
optimized_cot = optimizer.compile(CoT2(), trainset=dataset2)

100%|██████████████████████████████████████████████████████████████████████████████| 13/13 [1:49:12<00:00, 504.04s/it]


Bootstrapped 11 full traces after 13 examples in round 0.


In [ ]:
optimized_cot.forward(question = "Explain different types of Lighting mode?", context = text)

In [17]:
save_path = './cot_v2_11examples.json'
optimized_cot.save(save_path)

[('generate_answer2', Predict(StringSignature(context, question -> rationale, answer
    instructions='Think and Answer questions based on the context provided.'
    context = Field(annotation=str required=True json_schema_extra={'desc': 'contains relevant context', '__dspy_field_type': 'input', 'prefix': 'Context:'})
    question = Field(annotation=str required=True json_schema_extra={'desc': 'given question to be answer', '__dspy_field_type': 'input', 'prefix': 'Question:'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': 'generate detailed answer', '__dspy_field_type': 'output', 'prefix': 'Answer:'})
)))]


In [18]:
loaded_program = CoT2()
loaded_program.load(path=save_path)

In [19]:
# india_context = "India, officially the Republic of India (ISO: Bhārat Gaṇarājya),[21] is a country in South Asia. It is the seventh-largest country by area; the most populous country with effect from June 2023;[22][23] and from the time of its independence in 1947, the world's most populous democracy.[24][25][26] Bounded by the Indian Ocean on the south, the Arabian Sea on the southwest, and the Bay of Bengal on the southeast, it shares land borders with Pakistan to the west;[j] China, Nepal, and Bhutan to the north; and Bangladesh and Myanmar to the east. In the Indian Ocean, India is in the vicinity of Sri Lanka and the Maldives; its Andaman and Nicobar Islands share a maritime border with Thailand, Myanmar, and Indonesia.The capital of india is delhi."
loaded_program.generate_answer2(question = "Explain different types of Lighting mode?", context = text)

Prediction(
    rationale="Based on the provided context, I will answer your question.\n\nFrom the given regulations (E/ECE/324/Rev.1/Add.47/Rev. 12 and E/ECE/TRANS/505/Rev.1/Add.47/Rev. 12), we can identify different types of lighting modes as follows:\n\n1. **Dipped-beam headlamps**: These are the low beam headlights that provide a wider field of view for driving in urban areas or on roads with speed limits.\n\nAutomatic switching conditions for dipped-beam headlamps:\n\t* Less than 1,000 lux: ON within 2 seconds\n\t* Between 1,000 and 7,000 lux: at manufacturer's discretion (no specific",
    answer="Based on the provided context, I can identify different types of lighting modes as follows:\n\n1. **Dipped-beam headlamps**: These are the low beam headlights that provide a wider field of view for driving in urban areas or on roads with speed limits.\n\t* Automatic switching conditions:\n\t\t+ Less than 1,000 lux: ON within 2 seconds\n\t\t+ Between 1,000 and 7,000 lux: at manufacturer'